# base

> Includes the main datastructures for Nodes and Documents.

In [ ]:
#| default_exp base

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from abc import ABC, abstractmethod
from nbdev_rag.utils import *
import uuid

In [ ]:
#| export
class BaseNode(ABC):
    """
    Lowest level abstraction for storing interrelated pieces of information, building block for other types of nodes. 
    """
    def __init__(self, metadata, model_context, prev_node=None, next_node=None, parent_node=None, child_node=[], embedding=[]):
        self.metadata = metadata
        self.model_context = model_context
        self.prev_node = prev_node  # In llama index they have relationships in one dict. I can try it.
        self.next_node = next_node
        self.parent_node = parent_node
        self.child_node = child_node
        self.embedding = embedding
        self.id = self.__set_id()

    def __repr__(self):
        return f"BaseNode(id={self.id}, metadata={self.metadata}, prev_node={self.prev_node}, next_node={self.next_node}, parent_node={self.parent_node}, child_node={self.child_node})"

    def __set_id(self):
        return uuid.uuid4()

    def create_embedding(self):
        # Create embedding with the specific content inside.
        pass
    @abstractmethod
    def get_embedding(self):
        # Return the embeddings stored
        pass

In [ ]:
#| export

class TextNode(BaseNode): #Add hash to verify content uniqueness
    """Class for creating chunks of Text that contain additional information like relationships of metadata, inheritance from
    BaseNode but geared specifically towards text"""
    def __init__(self, text, model_context, metadata, prev_node = None, next_node = None, parent_node = None, child_node = [], embedding = [], auto_embed = True):
        super().__init__(metadata, model_context, prev_node, next_node, parent_node, child_node, embedding)
        self.text = text
        self.model_context = model_context
        self.embedding = None
        if auto_embed == True:
            self.create_embedding()
        self.hash = self.__calculate_hash()

    def __repr__(self):
        return f"TextNode(id = {self.id},text = {self.text},metadata = {self.metadata}, prev_node = {self.prev_node}, next_node = {self.next_node}, parent_node = {self.parent_node}, child_node = {self.child_node})"
    
    def create_embedding(self):
        if self.embedding == None:
            self.embedding = self.model_context.embedding.encode([self.text], normalize_embeddings = True) #Huggingface sentence transformers. #TODO: Generalize this, create wrapper.Embedding model would have get and create embedding.
    def get_embedding(self):
        if self.embedding is None:
            raise ValueError("embedding not set.")
        return self.embedding
        #Embedding for text. Try one for image.

    def __calculate_hash(self):
        return hash_input(f"{self.text}{self.metadata}{self.prev_node}{self.next_node}{self.parent_node}{self.child_node}")

In [ ]:
#| export

class Document(BaseNode): #A document is a collection of nodes #Add hash to verify content uniqueness
    #Also can be the source of information.
    """
    Class that serves as a way to group information that comes from different sources intended to be stored or integrated with other services
    """
    def __init__(self, metadata = {}, name = None, text = None, prev_node = None, next_node = None, parent_node = None, child_node = [], embedding = []):
        super().__init__(metadata, prev_node, next_node, parent_node, child_node, embedding)
        self.nodes = []
        self.text = text
        self.name = name
        self.hash = self.__calculate_hash()
        #Maybe add some extra info like info_source that can help storing directly in db.
        
    def __call__(self, nodes):
        self.nodes = nodes
        self.hash = self.__calculate_hash()
        
    def __repr__(self):
        return f"Document(id = {self.id}, name = {self.name}, metadata = {self.metadata}, n_nodes = {len(self.nodes)})"
    
    def __calculate_hash(self):
        return hash_input(f"{self.name}{self.metadata}{self.nodes}{self.prev_node}{self.next_node}{self.parent_node}{self.child_node}")

    def get_embedding(self):
        #Return the embedding of the nodes
        pass
    
    def create_nodes_from_doc(self,model_context, category = 'FILE', chunk_size = 1024):
        nodes = []
        chunked_text = self.__chunk_text(chunk_size)
        existing_metadata = self.metadata
        for i, text in enumerate(chunked_text):
            if i == 0:
                 prev_node = None
                 next_node = None
                 node_metadata = {**existing_metadata,**{'category' : category, 'node_height': 0, 'node_length':1}}
                 nodes.append(TextNode(text = text, metadata = node_metadata, model_context = model_context))
            else:
                 node_metadata = {**existing_metadata,**{'category' : category, 'node_height': 0, 'node_length':1}}
                 node = TextNode(text = text, metadata = node_metadata, model_context = model_context, prev_node =  nodes[i - 1].id)
                 nodes.append(node)
                 nodes[i - 1].next_node = node.id
        return nodes
        

    def __chunk_text(self, chunk_size = 1024): #Make with other types of text representations like dataframes. Initial version.
        chunked_text = []
        text = self.text
        for i in range(0, len(text), chunk_size):
            chunked_text.append(text[i:chunk_size + i])
        return chunked_text


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()